In [1]:
import open3d as o3d
import numpy as np
import math
import copy

In [2]:
def load_point_cloud(path):
    """
    Description
    -----------
        read point cloud

    Parameters
    ----------
        path: str

    Returns
    -------
        o3d.geometry.PointCloud
    """
    cloud = o3d.io.read_point_cloud(path)
    return cloud


In [3]:
from pyquaternion import Quaternion


def matrix2rotation(matrix):
    return matrix[:3, :3]


def matrix2qua(matrix):
    qua = Quaternion(matrix=matrix2rotation(matrix))
    return qua


def matrix2trans(matrix):
    return np.array(matrix[:3, 3:].transpose())[0]


def matrix2pose(matrix):
    qua = matrix2qua(matrix)
    translation = matrix2trans(matrix)
    return np.hstack((translation, qua.elements))


def quatrans2matrix(qua, trans):
    rotaion_matrix = qua.rotation_matrix
    trans_matrix = np.array([trans]).transpose()
    tf_m = np.hstack((rotaion_matrix, trans_matrix))
    m = np.mat([[0, 0, 0, 1]])
    tf_m = np.vstack((tf_m, m))

    return tf_m


def pose2matrix(pose):
    w, x, y, z = pose[3:]
    qua = Quaternion(w=w, x=x, y=y, z=z)
    trans = pose[:3]
    return quatrans2matrix(qua, trans)


def translation2matrix(trans):
    qua = Quaternion(w=1, x=0, y=0, z=0)
    return quatrans2matrix(qua, trans)


def qua2matrix(qua):
    trans = [0, 0, 0]
    return quatrans2matrix(qua, trans)

In [4]:
def get_picked_indices(pcd):
    vis = o3d.visualization.VisualizerWithEditing()
    vis.create_window(window_name="Resgistration: Pick Corresponding Points")
    vis.add_geometry(pcd)
    vis.run()
    vis.destroy_window()
    return vis.get_picked_points()


def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries(
        [source_temp, target], window_name="Registration Visualization"
    )
    return source_temp + target


# def icp(source, target, icp_threshold):
#     print('Please press Shift and click the feature points in the same order, press Q to finish')
#     picked_id_source = get_picked_indices(source)
#     print('picked_id_source: ', picked_id_source)
#     return picked_id
        
#     while True:
        
#         print('Please press Shift and click the feature points in the same order, press Q to finish')
#         picked_id_source = get_picked_indices(source)
#         print('picked_id_source: ', picked_id_source)
        
#         print('Please press Shift and click the feature points, press Q to finish')
#         picked_id_target = get_picked_indices(target)
#         print('picked_id_target: ', picked_id_target)

#         if len(picked_id_source) != len(picked_id_target):
#             print("The number of corresponding points are not the same, please re-pick")
#         else:
#             break

#     corr = np.zeros((len(picked_id_source), 2))
#     corr[:, 0] = picked_id_source
#     corr[:, 1] = picked_id_target

#     p2p = o3d.registration.TransformationEstimationPointToPoint()
#     trans_init = p2p.compute_transformation(
#         source, target, o3d.utility.Vector2iVector(corr)
#     )

#     reg_p2p = o3d.registration.registration_icp(
#         source,
#         target,
#         icp_threshold,
#         trans_init,
#         o3d.registration.TransformationEstimationPointToPoint(),
#     )

#     transform = reg_p2p.transformation
#     print("Transformation Matrix: \n{}\n".format(transform))

#     final_cloud = draw_registration_result(
#         source, target, transform)
    
#     return transform

In [5]:
def load_mesh_model(path):
    """
    Description
    -----------
        read mesh model

    Parameters
    ----------
        path: str

    Returns
    -------
        o3d.geometry.TriangleMesh
    """
    mesh = o3d.io.read_triangle_mesh(path)
    return mesh

def sample_mesh_to_cloud(mesh, sample_number=100000):
    """
    Description
    -----------
        process the cabinet mesh model, sample it as point cloud

    Parameters
    ----------
        mesh: o3d.geometry.TriangleMesh
        sample_number: int

    Returns
    -------
        o3d.geometry.PointCloud
    """
    mesh.compute_vertex_normals()
    pcd = mesh.sample_points_uniformly(number_of_points=sample_number)
    pcd.paint_uniform_color([1, 0.706, 0])  # yellow
    return pcd

In [29]:
path = "/home/bot/dev/loading_projects/onsite_data/calibration_frame_2.STL"
mesh = load_mesh_model(path)
target = sample_mesh_to_cloud(mesh, 500000)

# add sign on front
add_sign = True
if add_sign:
    sign_on_front = o3d.geometry.TriangleMesh.create_box(width=0.01, height=0.01, depth=0.01)
    sign_cloud = sample_mesh_to_cloud(sign_on_front, 1000)
    sign_cloud.transform(pose2matrix([-0.5, 0, 1.5, 1, 0, 0, 0]).A)
    target =  target + sign_cloud


target.transform(pose2matrix([2.98, 0.78, 0.07, 1, 0, 0, 0]).A)

o3d.visualization.draw_geometries([target])

In [14]:
target = load_point_cloud("/home/bot/dev/loading_projects/onsite_data/0115_calibration/left.pcd")
source = load_point_cloud("/home/bot/dev/loading_projects/onsite_data/0115_calibration/right.pcd")

In [7]:
source_ids = get_picked_indices(source)
print(len(source_ids))
print(source_ids)

9
[51102, 51408, 69115, 73538, 82471, 81839, 75172, 74868, 90771]


In [8]:
target_ids = get_picked_indices(target)
print(len(target_ids))
print(target_ids)

9
[7564, 9987, 23359, 31453, 38466, 38499, 33860, 33889, 53086]


In [11]:
corr = np.zeros((len(source_ids), 2))
corr[:, 0] = source_ids
corr[:, 1] = target_ids

p2p = o3d.registration.TransformationEstimationPointToPoint()
trans_init = p2p.compute_transformation(
    source, target, o3d.utility.Vector2iVector(corr)
)
print(trans_init)

[[ 0.99811683  0.06025602 -0.01148917 -0.05445297]
 [-0.05716085  0.98159289  0.1822307  -1.58863095]
 [ 0.02225818 -0.1812308   0.98318867  0.12819161]
 [ 0.          0.          0.          1.        ]]


In [12]:
draw_registration_result(source, target, trans_init)

geometry::PointCloud with 207169 points.

In [13]:
center_point = np.array([0,0,0])
min_bound = center_point - np.asarray([2, 2, 0])
max_bound = center_point + np.asarray([2, 2, 3.5])

cropbox = o3d.geometry.AxisAlignedBoundingBox(min_bound=min_bound, max_bound=max_bound)

source = source.crop(cropbox)
target = target.crop(cropbox)

In [15]:
reg_p2p = o3d.registration.registration_icp(
    source,
    target,
    0.005,
    trans_init,
    o3d.registration.TransformationEstimationPointToPoint(),
)

transform = reg_p2p.transformation
print("Transformation Matrix: \n{}\n".format(transform))

Transformation Matrix: 
[[ 0.99812778  0.06067389 -0.0077205  -0.071933  ]
 [-0.05821224  0.98110818  0.18449411 -1.59399038]
 [ 0.01876862 -0.18369927  0.98280329  0.13616869]
 [ 0.          0.          0.          1.        ]]



In [16]:
final_cloud = draw_registration_result(source, target, transform)

In [17]:
pose = matrix2pose(transform)
print(pose)

[-0.071933   -1.59399038  0.13616869  0.9952436  -0.09248826 -0.00665393
 -0.02986358]
